## Task 4. Data processing 
Preparation of data for Task 3.

In [1]:
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from skimage import io, color
from skimage.transform import resize
import numpy as np
import os
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import numpy as np
import os
import cv2
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
from skimage.feature import local_binary_pattern
import matplotlib.pyplot as plt

In [2]:
train_path = "C:\\Users\\Tomi\\Desktop\\archive\\fruits-360_dataset\\fruits-360\\Training"
test_path = "C:\\Users\\Tomi\\Desktop\\archive\\fruits-360_dataset\\fruits-360\\Test"

In [3]:
def load_fruits360_dataset(train_dir, test_dir, img_size=(100, 100), max_per_class=None, fruit_name=[]):
    images = []
    labels = []
    class_dict = {}
    class_counter = 0

    # Function to load images from a directory
    def load_images_from_dir(directory, label, max_count=None):
        count = 0

        for img_name in os.listdir(directory):
            if max_count and count >= max_count:
                break
            img_path = os.path.join(directory, img_name)
            img = cv2.imread(img_path)
            if img is not None:
                img = cv2.resize(img, img_size)
                images.append(img)
                labels.append(label)
                count += 1
    
    # Load images from the training directory
    for class_name in sorted(os.listdir(train_dir)):
        if class_name in fruit_name:
            class_path = os.path.join(train_dir, class_name)
            if class_name not in class_dict:
                class_dict[class_name] = class_counter
                class_counter += 1
            load_images_from_dir(class_path, class_dict[class_name], max_per_class)

    # Load images from the test directory
    for class_name in sorted(os.listdir(test_dir)):
        if class_name in fruit_name:
            class_path = os.path.join(test_dir, class_name)
            if class_name not in class_dict:
                class_dict[class_name] = class_counter
                class_counter += 1
            load_images_from_dir(class_path, class_dict[class_name], max_per_class)

    images = np.array(images)
    labels = np.array(labels)
    return images, labels, class_dict

In [4]:
# Specify the paths to the training and test dataset directories
train_dir = train_path
test_dir = test_path
max_per_class = 164 # Set the desired number of images per class
fruit_ = ["Grape White","Grape Pink","Grape Blue"]

# Load the dataset
images, labels, class_dict = load_fruits360_dataset(train_dir,test_dir, max_per_class=max_per_class,fruit_name=fruit_)

In [5]:
len(labels)

984

In [6]:
# Function to extract features from an image
def extract_features(image):
    # Color features (mean and std of each channel)
    mean_color = np.mean(image, axis=(0, 1))
    std_color = np.std(image, axis=(0, 1))
    

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray_hist = np.histogram(gray, bins=256)[0]
    gray_hist = gray_hist[:-6]
    gray_mean = np.mean(gray_hist)
    edges = cv2.Canny(gray, 100, 200)
    edge_density = np.sum(edges > 0) / edges.size
    
    # Texture features (LBP)
    lbp = local_binary_pattern(gray, P=8, R=1, method="uniform")
    lbp_hist = np.histogram(lbp.ravel(), bins=np.arange(0, 11), density=True)[0]
    lbp_entropy = -np.sum(lbp_hist * np.log(lbp_hist + 1e-10))
    
    return np.hstack([mean_color, std_color, gray_mean,edge_density, lbp_entropy])


# Extract features from each image
features = []
for img in images:
    feature_vector = extract_features(img)
    features.append(feature_vector)

In [7]:
column_names = ['mean_color_r', 'mean_color_g', 'mean_color_b','std_color_r', 'std_color_g', 'std_color_b','gray_mean','edge_density','lbp_entropy']
features_df = pd.DataFrame(features, columns=column_names)
features_df['label'] = labels

In [8]:
features_df

,mean_color_r,mean_color_g,mean_color_b,std_color_r,std_color_g,std_color_b,gray_mean,edge_density,lbp_entropy,label
0,81.3879,77.5498,76.5709,85.860866,86.667149,87.172842,32.936,0.0390,1.995038,0
1,82.7306,77.9803,76.4728,85.410408,86.341648,86.934091,33.056,0.0378,2.021271,0
2,78.8807,73.9551,72.3358,82.481815,83.359455,83.960948,33.700,0.0383,2.027392,0
3,79.0510,74.1712,72.5618,82.664205,83.501287,84.089438,33.644,0.0372,2.010986,0
4,78.5307,73.6627,71.8714,82.177894,82.946881,83.614232,33.796,0.0335,2.015285,0
...,...,...,...,...,...,...,...,...,...,...
979,160.5034,191.9486,190.5294,72.834488,47.252466,47.224957,26.860,0.0364,1.996450,2
980,160.4089,191.9957,190.6816,72.746627,47.094648,47.036614,26.884,0.0364,2.003313,2
981,160.1665,191.9310,190.5030,72.796792,47.112421,47.126638,26.888,0.0379,2.003467,2
982,160.3646,191.9091,190.7422,72.609460,47.122080,47.033564,26.892,0.0344,2.013670,2


In [9]:
features_df.to_csv('grape_features.csv', index=False)
print("DataFrame saved as 'grape_features.csv'")

DataFrame saved as 'grape_features.csv'


In [10]:
X = features_df.drop('label', axis=1) 
y = features_df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Training features shape:", X_train.shape)
print("Testing features shape:", X_test.shape)
print("Training labels shape:", y_train.shape)
print("Testing labels shape:", y_test.shape)

Training features shape: (787, 9)
Testing features shape: (197, 9)
Training labels shape: (787,)
Testing labels shape: (197,)
